In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
def _pick_number(prob: float) -> str:
    choices = [0, 1]
    probabilities = [1 - prob, prob]
    return random.choices(choices, weights=probabilities)[0]

In [3]:
def _generate_random(mean:int, sd:int) -> int:
    random_number = np.abs(np.random.normal(mean, sd))
    return int(random_number)

In [4]:
def generate_random_arr(rows, prob, mean:int=40, sd:int=40) -> list:
    arr = []
    count = 0
    while count < rows:
        val = _pick_number(prob)
        n = _generate_random(mean, sd)
        arr.extend([val for i in range (n)])
        count += n
    return arr[:rows]

In [5]:
def generate_worker_df(worker_id: int, worker_settings: dict) -> pd.DataFrame:
    start = datetime(2023, 5, 1, 9, 0, 0)
    end = datetime(2023, 5, 1, 18, 0, 0)
    num_rows = (start-end).seconds // 5
    df_dict = {"timestamp": [], "worker_id": [], "ppe": [], "danger": [], "safety_harness": []}
    
    # Generate timestamp
    curr = start
    for i in range(num_rows):
        df_dict["timestamp"].append(curr)
        curr += timedelta(seconds=5 * i)

    # Generate worker_id
    df_dict["worker_id"] = [worker_id for i in range(num_rows)]

    # Generate ppe, danger, safety_harness
    df_dict["ppe"] = generate_random_arr(num_rows, worker_settings["prob_ppe"])
    df_dict["danger"] = generate_random_arr(num_rows, worker_settings["prob_danger"])
    df_dict["safety_harness"] = generate_random_arr(num_rows, worker_settings["prob_safety_harness"])

    df = pd.DataFrame(df_dict)
    return df

In [6]:
worker_settings_dict = {
    1: {
        "prob_ppe": 0.8,
        "prob_danger": 0.2,
        "prob_safety_harness": 0.8,
    },
    2: {
        "prob_ppe": 0.2,
        "prob_danger": 0.8,
        "prob_safety_harness": 0.8,
    },
    3: {
        "prob_ppe": 0.8,
        "prob_danger": 0.8,
        "prob_safety_harness": 0.2,
    }
}

In [7]:
# Generate data
all_df = None
for worker_id, worker_settings in worker_settings_dict.items():
    worker_df = generate_worker_df(worker_id, worker_settings)
    all_df = worker_df if all_df is None else pd.concat([all_df, worker_df], axis=0, ignore_index=True)

In [8]:
# Check df
all_df

timestamp  worker_id  ppe  danger  safety_harness
0     2023-05-01 09:00:00          1    1       0               1
1     2023-05-01 09:00:00          1    1       0               1
2     2023-05-01 09:00:05          1    1       0               1
3     2023-05-01 09:00:15          1    1       0               1
4     2023-05-01 09:00:30          1    1       0               1
...                   ...        ...  ...     ...             ...
32395 2032-07-23 22:31:15          3    1       1               1
32396 2032-07-24 13:30:50          3    1       1               1
32397 2032-07-25 04:30:30          3    1       1               1
32398 2032-07-25 19:30:15          3    1       1               1
32399 2032-07-26 10:30:05          3    1       1               1

[32400 rows x 5 columns]

In [9]:
# Save df
all_df.to_csv("data/file.csv")

In [10]:
# Visualize
all_df[all_df["worker_id"]==2]["ppe"][0:1000].plot()

<Axes: >